In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score

quadratic_kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

### 1. Load Data

In [5]:
train_tabular_df = pd.read_csv('train.csv')
test_tabular_df = pd.read_csv('test.csv')

### 2. Preprocess Data

In [7]:
def getRescuerIDCountMap(data):
    rescuerid_counts = data['RescuerID'].value_counts().to_dict()
    return rescuerid_counts

def encodeRescuerIDCount(data, mapping):
    dataset = data.copy()
    dataset['RescuerID_Count'] = dataset['RescuerID'].map(mapping).fillna(0)
    return dataset

def createRescuerIDCount(data):
    # RescuerID Counts
    dataset = data.copy()
    rescuerid_counts = getRescuerIDCountMap(dataset)
    dataset = encodeRescuerIDCount(dataset, rescuerid_counts)
    return dataset, rescuerid_counts

def createDummyVariables(data):
    dataset = data.copy()
    # fee 0 dummy
    dataset['Flag_Fee_0'] = np.nan
    dataset['Flag_Fee_0'] = dataset['Flag_Fee_0'].mask(dataset['Fee'] == 0, 1)
    dataset['Flag_Fee_0'] = dataset['Flag_Fee_0'].fillna(0)

    # breed dummy variables
    mixed_breed_id = 307

    # pure breeds are where Breed1 == Breed2
    pure_breed_cond = (dataset['Breed1'] != mixed_breed_id) & (dataset['Breed1'] == dataset['Breed2'])
    dataset['Flag_Pure_Breed'] = np.nan
    dataset['Flag_Pure_Breed'] = dataset['Flag_Pure_Breed'].mask(pure_breed_cond, 1)
    dataset['Flag_Pure_Breed'] = dataset['Flag_Pure_Breed'].fillna(0)

    # we define mixed breeds as Breed1 or Breed2 being Mixed-Breed, or Breed1 != Breed2 where Breed1 and Breed2 are not Mixed-Breed
    mixed_breed_cond = ((dataset['Breed1'] == 307) | (dataset['Breed2'] == 307)) | ((dataset['Breed1'] != dataset['Breed2']) & 
                                                                                             (dataset['Breed2'] != 0) & 
                                                                                             (dataset['Breed2'] != 307)& 
                                                                                             (dataset['Breed1'] != 307))
    dataset['Flag_Mixed_Breed'] = np.nan
    dataset['Flag_Mixed_Breed'] = dataset['Flag_Mixed_Breed'].mask(mixed_breed_cond, 1)
    dataset['Flag_Mixed_Breed'] = dataset['Flag_Mixed_Breed'].fillna(0)
    
    # age < 3 months
    dataset['Flag_Age_le_3'] = np.nan
    dataset['Flag_Age_le_3'] = dataset['Flag_Age_le_3'].mask(dataset['Age'] <= 3, 1)
    dataset['Flag_Age_le_3'] = dataset['Flag_Age_le_3'].fillna(0)
    return dataset

def standardiseVariables(data):
#    std_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', 'RescuerID_Count']
    std_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt']
    dataset = data.copy()
    dataset[[c + "_std" for c in std_cols]] = StandardScaler().fit_transform(dataset[std_cols])
    return dataset

def oneHotEncode(data):
    dummy_cols = ['Type','Gender','Health','FurLength','State','Color1','Color2','Color3']
    dataset = data.copy()
    return pd.get_dummies(dataset, columns=dummy_cols)

def getTargetReEncodingMap(data, col, response):
    col_map = {k: idx+1  for idx, k in enumerate(data.groupby([col])[response].mean().sort_values().index)}
    return col_map

def targetReEncodeFeature(data, col, mapping):
    dataset = data.copy()
    dataset[col + '_target_encode'] = dataset[col].map(mapping).fillna(0)
    return dataset

def targetReEncodeFeatures(data, cols, response):
    dataset = data.copy()
    mappings = dict()
    for col in cols:
        mapping = getTargetReEncodingMap(dataset, col, response)
        mappings[col] = mapping
        dataset = targetReEncodeFeature(dataset, col, mapping)
    return dataset, mappings

def getTargetEncodingMap(data, col, response):
    target_map = data.groupby([col])[response].mean().to_dict()
    return target_map

def targetEncodeFeature(data, col, mapping):
    # yes i know this is the same as above but im literally so lazy so ctrl c + v
    dataset = data.copy()
    dataset[col + '_target_encode'] = dataset[col].map(mapping).fillna(0)
    return dataset

def targetEncodeFeatures(data, cols, response):
    # yes i know this is the same as above but im literally so lazy so ctrl c + v
    dataset = data.copy()
    mappings = dict()
    for col in cols:
        mapping = getTargetEncodingMap(dataset, col, response)
        mappings[col] = mapping
        dataset = targetEncodeFeature(dataset, col, mapping)
    return dataset, mappings

def dropNonFeatureColumns(data):
    non_feature_cols = [
        'PetID',
        'Name',
        'RescuerID',
        'Age',
        'Quantity',
        'Fee',
        'PhotoAmt',
        'VideoAmt',
        'Description',
        'Sterilized',
        'Vaccinated',
        'Dewormed',
        'MaturitySize'
    ]
    dataset = data.copy()
    return dataset.drop(columns=non_feature_cols)

def preProcessTrain(data):
    mappings = dict()
    data, mappings['RescuerID'] = createRescuerIDCount(data)
    data = createDummyVariables(data)
    data = standardiseVariables(data)
    data = oneHotEncode(data)
    data, target_remappings = targetReEncodeFeatures(data,
                                                    cols=['Sterilized', 'Vaccinated', 'Dewormed', 'MaturitySize'],
                                                    response='AdoptionSpeed')
    data, target_mappings = targetEncodeFeatures(data,
                                                cols=['Breed1', 'Breed2'],
                                                response='AdoptionSpeed')
    mappings.update(target_remappings)
    mappings.update(target_mappings)
    data = dropNonFeatureColumns(data)
    return data, mappings

def preProcessTest(data, mappings, train_cols):
    train_cols = list(train_cols)
    train_cols.remove('AdoptionSpeed')
    data = data.fillna(0)
    data = encodeRescuerIDCount(data, mappings['RescuerID'])
    data = createDummyVariables(data)
    data = standardiseVariables(data)
    data = oneHotEncode(data)
    data = targetReEncodeFeature(data, 'Sterilized', mappings['Sterilized'])
    data = targetReEncodeFeature(data, 'Vaccinated', mappings['Vaccinated'])
    data = targetReEncodeFeature(data, 'MaturitySize', mappings['MaturitySize'])
    data = targetReEncodeFeature(data, 'Dewormed', mappings['Dewormed'])
    data = targetEncodeFeature(data, 'Breed1', mappings['Breed1'])
    data = targetEncodeFeature(data, 'Breed2', mappings['Breed2'])
    data = dropNonFeatureColumns(data)
    data = data[train_cols]
    return data

In [8]:
train, mappings = preProcessTrain(train_tabular_df)
test = preProcessTest(test_tabular_df, mappings, train.columns)

### 3. Tuning Hyperparameters for SVM Model with RBF Kernel

In [15]:
def splitDataXY(dataset, response_column):
    temp = dataset.copy()
    Y_train = temp[response_column]
    X_train = temp.drop(columns=[response_column])
    return X_train.to_numpy(), Y_train.to_numpy()

In [16]:
X_train, Y_train = splitDataXY(train, 'AdoptionSpeed')

 Hyperparameters:
1. `C`: 
- C is the penalty parameter, which represents misclassification or error term, used to maintain regularization. 
- The misclassification or error term tells the SVM optimization how much error is bearable. This is how you can control the trade-off between decision boundary and misclassification term. 
- A smaller value of C creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane.
2. `kernel`: 
- Kernel transforms the given dataset input data into the required form - linear, polynomial, and radial basis function (RBF). 
- Polynomial and RBF are useful for non-linear hyperplane. Polynomial and RBF kernels compute the separation line in the higher dimension. 
3. Gamma `γ` : 
- A lower value of Gamma will loosely fit the training dataset, whereas a higher value of gamma will exactly fit the training dataset, which causes over-fitting.

In [20]:
param_grid = {'C': [0.1, 1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(svm.SVC(),
                    param_grid,
                    verbose = 3,
                    scoring=quadratic_kappa_scorer)

In [21]:
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  45.0s
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  42.6s
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  43.1s
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  43.6s
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  47.9s
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  48.8s
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  46.4s
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  41.2s
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  42.0s
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  41.7s
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time=  40.9s
[CV 2/5] END ..................C=0.1, gamma=0.0

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 500, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring=make_scorer(cohen_kappa_score, weights=quadratic),
             verbose=3)

In [22]:
print('Best Parameters:', grid.best_params_)
print('Best Quadratic Kappa', grid.best_score_)

Best Parameters: {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
Best Quadratic Kappa 0.35471099693582453


### 4. Train

In [27]:
svm_model = svm.SVC(C=1000, gamma=0.0001, kernel='rbf')

In [28]:
svm_model.fit(X_train, Y_train)

SVC(C=1000, gamma=0.0001)

In [34]:
train_pred = svm_model.predict(X_train)

### 5. Evaluate

In [37]:
print('Accuracy: ', metrics.accuracy_score(train_pred, Y_train))

Accuracy:  0.49896618421930233


In [38]:
print('Evaluation metric: ', cohen_kappa_score(train_pred, Y_train, weights='quadratic'))

Evaluation metric:  0.43731903386610527


### 6. Adoption Speed Prediction

In [32]:
clf_pred = pd.DataFrame()
clf_pred['PetID'] = test_tabular_df['PetID']
clf_pred['AdoptionSpeed'] = svm_model.predict(test)

In [33]:
clf_pred

,PetID,AdoptionSpeed
0,e2dfc2935,4
1,f153b465f,4
2,3c90f3f54,2
3,e02abc8a3,4
4,09f0df7d1,4
...,...,...
3967,ae57f8d52,4
3968,83432904d,4
3969,399013029,4
3970,fd80b8c80,4
